In [11]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import sklearn
import os
import tensorflow as tf
!pip install efficientnet
import efficientnet.tfkeras  as efn

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


# load data

In [7]:
path=r"C:\work\freelancing\plants-collage-project\PlantVillage-Dataset\raw\color"

In [21]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        path,
        target_size=(256, 256),
        batch_size=64,
        class_mode='categorical')

Found 54305 images belonging to 38 classes.


# model

In [12]:
model = tf.keras.models.Sequential([efn.EfficientNetB6(include_top=False, weights='imagenet',input_shape=(256, 256, 3),pooling='avg'),
                                   tf.keras.layers.Dense(1024,activation="relu"),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(512,activation="relu"),
                                    tf.keras.layers.Dense(256,activation="relu"),
                                    tf.keras.layers.Dense(128,activation="relu"),
                                    tf.keras.layers.Dense(38,activation="softmax")])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b6 (Model)      (None, 2304)              40960136  
_________________________________________________________________
dense (Dense)                (None, 1024)              2360320   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_4 (Dense)              (None, 38)                4

In [14]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=['accuracy'])

In [22]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000,
        epochs=1,
        validation_steps=800)

TypeError: 'NoneType' object is not callable

# Testing

In [37]:
model.evaluate(X_test,y_test)

768/768 [==============================] - 14s 18ms/step


[588.1274757385254, 0.4739583432674408]

In [38]:
model.save("model-v2.h5")

# load model

In [11]:
from keras.models import load_model
model=load_model(r"C:\work\freelancing\plants-collage-project\models\model-v2.h5")
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 516128)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 38)                19612902  
Total params: 19,613,798
Trainable params: 19,613,798
Non-trainable params: 0
_________________________________________________________________


# save to .pb
# https://blog.metaflow.fr/tensorflow-how-to-freeze-a-model-and-serve-it-with-a-python-api-d4f3596b3adc

In [5]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

In [7]:
from keras import backend as K
import tensorflow as tf
# Create, compile and train model...

frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 23 variables.
INFO:tensorflow:Converted 23 variables to const ops.


In [8]:
help(tf.train.write_graph)

Help on function write_graph in module tensorflow.python.framework.graph_io:

write_graph(graph_or_graph_def, logdir, name, as_text=True)
    Writes a graph proto to a file.
    
    The graph is written as a text proto unless `as_text` is `False`.
    
    ```python
    v = tf.Variable(0, name='my_variable')
    sess = tf.compat.v1.Session()
    tf.io.write_graph(sess.graph_def, '/tmp/my-model', 'train.pbtxt')
    ```
    
    or
    
    ```python
    v = tf.Variable(0, name='my_variable')
    sess = tf.compat.v1.Session()
    tf.io.write_graph(sess.graph, '/tmp/my-model', 'train.pbtxt')
    ```
    
    Args:
      graph_or_graph_def: A `Graph` or a `GraphDef` protocol buffer.
      logdir: Directory where to write the graph. This can refer to remote
        filesystems, such as Google Cloud Storage (GCS).
      name: Filename for the graph.
      as_text: If `True`, writes the graph as an ASCII proto.
    
    Returns:
      The path of the output proto file.



In [10]:
tf.train.write_graph(frozen_graph, "", "my_model.pb", as_text=False)

'my_model.pb'

# save to TFllite

In [13]:
#from_keras_model_file
file=r"C:\work\freelancing\plants-collage-project\models\model-v2.h5"
converter = tf.lite.TFLiteConverter.from_keras_model_file(file)
# Convert the model
tflite_model = converter.convert()
# Create the tflite model file
tflite_model_name = "mymodel.tflite"
open(tflite_model_name, "wb").write(tflite_model)

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 4 variables.
INFO:tensorflow:Converted 4 variables to const ops.


78456504